In [1]:
import pandas as pd
import re

In [2]:
msft = pd.read_excel('MicrosoftReviews.xlsx')
intel = pd.read_excel('IntelReviews.xlsx')
cisco = pd.read_excel('CiscoReviews.xlsx')

In [3]:
msft['Company'] = 'Microsoft'
intel['Company'] = 'Intel'
cisco['Company'] = 'Cisco'

In [4]:
data = pd.concat([msft, intel, cisco])
data.reset_index(inplace=True, drop=True)
data

,Headline,Status,Date,Pros,Cons,Company
0,Best Place To Work,"Current Employee, less than 1 year","Apr 14, 2021","Benefits, D&I, work life balance",I am home at Microsoft.,Microsoft
1,Thoughts after 10 years....,"Current Employee, more than 10 years","Jan 28, 2013","1. If you love tech, this is a great place. ...",Brand on Your Resume: After many years of los...,Microsoft
2,"low RSUs, salary, bonuses and career growth","Current Employee, less than 1 year","Apr 5, 2021","Great work life balance, good health benefits....","Average Engineers, Scientists and PMs with not...",Microsoft
3,"Slow movement, massive rewards","Current Employee, more than 5 years","Feb 22, 2021",Pay and compensation regularly above market. R...,"Benefits standard for tech, but on the lower r...",Microsoft
4,Supports personal career development,"Former Employee, less than 1 year","Mar 18, 2021",Ample on-line resources for everything from le...,I never thought laptops would be an issue but ...,Microsoft
...,...,...,...,...,...,...
61239,An ambitious company with its heart in the rig...,"Current Employee, less than 1 year","Jun 13, 2008",An ambitious company with its heart in the rig...,"Despite working for a networking company, comm...",Cisco
61240,Treat your front-line customer-facing staff be...,"Current Employee, less than 1 year","Jun 12, 2008",Breadth of technology - fantastic being able t...,Have no life - submitting numbers on Sunday af...,Cisco
61241,Quick,"Former Employee, less than 1 year","May 2, 2008",Better career path._x000D_\nRegional positions...,number of resources needs to increase,Cisco
61242,Hunt for the best company to work for,"Current Employee, less than 1 year","May 1, 2008","Best work place, Best recognition, Best Teams ...",there are no down sides,Cisco


In [5]:
data.isna().sum()

Headline    102
Status        0
Date          0
Pros          0
Cons          0
Company       0
dtype: int64

In [6]:
keywords = ['disability', 'disabled person', 'disabilities', 'autism', 'accessibility', 'accessible', 'erg', 'employee resource group', 'inclusion', 'inclusivity', 
            'inclusive', 'disabled employee', 'impaired', 'vision', 'captioning', 'voice-to-text', 'voice to text', 
            'discriminate', 'discrimination', 'equal employment', 'equal opportunity', 'supportive', 'mental', 'accommodation', 
            'accommodating', 'diversity', 'diverse', 'ndeam', 'national disability employment awareness month', 'a11y', 'abilities',
           'empowered', 'empowerment', 'paralysed', 'paralysis', 'transcribed', 'transcription', 'speech recognition', 'blind', 
           'blindness', 'culture', 'invisible', 'screen reader', 'representation', 'therapy', 'disorder', 'dei', 'wheelchair', 
            'hearing', 'neurodiverse', 'assistive', 'deafness', 'hiring', 'job', 'interview', 'employment', 'employer', 'recruit']

disability = ['disability', 'disabled person', 'autism', 'disabilities', 'accessibility', 'accessible', 'disabled employee', 'impaired', 'vision', 'captioning', 
              'voice-to-text', 'voice to text',  'mental', 'accommodation', 'accommodating', 'ndeam', 'national disability employment awareness month', 
              'a11y', 'abilities', 'paralysed', 'paralysis', 'transcribed', 'transcription', 'speech recognition', 'blind', 
           'blindness', 'invisible', 'screen reader',  'therapy', 'disorder', 'dei', 'wheelchair', 
            'hearing', 'neurodiverse', 'assistive', 'deafness']

culture = ['erg', 'employee resource group', 'inclusion', 'inclusivity', 'supportive', 'diversity', 'diverse', 'empowered', 'empowerment',
            'inclusive', 'discriminate', 'discrimination', 'equal employment', 'equal opportunity', 'culture', 'representation']

hiring = ['hiring', 'job', 'interview', 'employment', 'employer', 'recruit']

In [7]:
def keys(str):
    key = ''
    for i in keywords:
        if re.search(r'\b' + i + r'\b', str):
            key +=  i + ' ' 
    return key 

def isRelevant(str):
    for i in keywords:
        #a = '\b' + i + '\b'
        if re.search(r'\b' + i + r'\b', str):
            return 1 
    return 0

def tags(str):
    labels = ''
    for i in disability:
        if re.search(r'\b' + i + r'\b', str):
            labels += 'disability' + ' '
            break
    for i in culture:
        if re.search(r'\b' + i + r'\b', str):
            labels += 'culture' + ' '
            break
    for i in hiring:
        if re.search(r'\b' + i + r'\b', str):
            labels += 'hiring' + ' '
            break
    return labels

In [8]:
relevance, key, labels = [], [], []
for i in range(len(data)):
    relevance.append(isRelevant(data.Pros[i]) or isRelevant(data.Cons[i]))
    key.append(keys(data.Pros[i]) or keys(data.Cons[i]))
    labels.append(tags(data.Pros[i]) or tags(data.Cons[i]))
data['Relevance'] = relevance
data['Keywords'] = key
data['Labels'] = labels

In [9]:
data['Relevance'].value_counts()

0    45773
1    15471
Name: Relevance, dtype: int64

In [10]:
data = data.loc[data['Relevance'] == True]
data

,Headline,Status,Date,Pros,Cons,Company,Relevance,Keywords,Labels
1,Thoughts after 10 years....,"Current Employee, more than 10 years","Jan 28, 2013","1. If you love tech, this is a great place. ...",Brand on Your Resume: After many years of los...,Microsoft,1,job,hiring
3,"Slow movement, massive rewards","Current Employee, more than 5 years","Feb 22, 2021",Pay and compensation regularly above market. R...,"Benefits standard for tech, but on the lower r...",Microsoft,1,inclusion diverse hiring,culture hiring
5,Tech company with a moral compass,"Current Employee, less than 1 year","Mar 15, 2021",Senior leadership has driven top-down culture ...,"Work-life balance; it's a fast-paced, target-d...",Microsoft,1,inclusion culture,culture
6,"Great Company, but only for Engineering Organi...","Current Employee, less than 1 year","Mar 1, 2021",- Excellent employee culture & groups to explo...,- Unless you are in the engineering organizati...,Microsoft,1,culture,culture
7,Microsoft is a great company to work for,"Current Employee, more than 3 years","Jan 13, 2021","Great benefits, great pay compared to a smalle...",It's a big company so don't expect to get to d...,Microsoft,1,diverse job,culture hiring
...,...,...,...,...,...,...,...,...,...
61233,Cisco is the best place to work for techies wh...,"Current Employee, less than 1 year","Jun 12, 2008",Access to the best technical resources and peo...,The HR department is a bit of a monster. I som...,Cisco,1,job,hiring
61234,"Good on Resume, not easy to advance","Current Employee, less than 1 year","Jun 26, 2008 -","International community, market leader and goo...","too much politics (culture), internal salaries...",Cisco,1,culture,culture
61236,Cisco is young. it is still a good work place ...,"Current Employee, less than 1 year","Jun 23, 2008",Cisco is the leading company in network. It sh...,Salary is not competitive and not increase qui...,Cisco,1,culture,culture
61238,Great place to work in the 90's!,"Former Employee, less than 1 year","Jun 15, 2008",Salary and benefits. In the past they empowere...,It WAS the best employer to work for in the 90...,Cisco,1,empowered,culture


In [11]:
data['Company'].value_counts()

Microsoft    6589
Cisco        5235
Intel        3647
Name: Company, dtype: int64

In [12]:
data = data.drop(['Relevance'], axis = 1)
data

,Headline,Status,Date,Pros,Cons,Company,Keywords,Labels
1,Thoughts after 10 years....,"Current Employee, more than 10 years","Jan 28, 2013","1. If you love tech, this is a great place. ...",Brand on Your Resume: After many years of los...,Microsoft,job,hiring
3,"Slow movement, massive rewards","Current Employee, more than 5 years","Feb 22, 2021",Pay and compensation regularly above market. R...,"Benefits standard for tech, but on the lower r...",Microsoft,inclusion diverse hiring,culture hiring
5,Tech company with a moral compass,"Current Employee, less than 1 year","Mar 15, 2021",Senior leadership has driven top-down culture ...,"Work-life balance; it's a fast-paced, target-d...",Microsoft,inclusion culture,culture
6,"Great Company, but only for Engineering Organi...","Current Employee, less than 1 year","Mar 1, 2021",- Excellent employee culture & groups to explo...,- Unless you are in the engineering organizati...,Microsoft,culture,culture
7,Microsoft is a great company to work for,"Current Employee, more than 3 years","Jan 13, 2021","Great benefits, great pay compared to a smalle...",It's a big company so don't expect to get to d...,Microsoft,diverse job,culture hiring
...,...,...,...,...,...,...,...,...
61233,Cisco is the best place to work for techies wh...,"Current Employee, less than 1 year","Jun 12, 2008",Access to the best technical resources and peo...,The HR department is a bit of a monster. I som...,Cisco,job,hiring
61234,"Good on Resume, not easy to advance","Current Employee, less than 1 year","Jun 26, 2008 -","International community, market leader and goo...","too much politics (culture), internal salaries...",Cisco,culture,culture
61236,Cisco is young. it is still a good work place ...,"Current Employee, less than 1 year","Jun 23, 2008",Cisco is the leading company in network. It sh...,Salary is not competitive and not increase qui...,Cisco,culture,culture
61238,Great place to work in the 90's!,"Former Employee, less than 1 year","Jun 15, 2008",Salary and benefits. In the past they empowere...,It WAS the best employer to work for in the 90...,Cisco,empowered,culture


In [13]:
data.to_excel('Relevant_Glassdoor.xlsx', header=True, index=False)